# K₇ Explicit Construction — CuPy + Pell Structure

**Objectif**: Construire K₇ explicitement et résoudre λ₁ × H* = 13 ou 14

---

## Nouveaux Insights

1. **Structure Pell**: 99² − 50×14² = 1
2. **√50 = [7; 14̄]** où 7 = dim(K₇), 14 = dim(G₂)
3. **λ₁ = 14/99 = [0; 7, 14]** — fraction continue pure
4. **K₇ hors TCS standard** (b₂=21 > 9) → définition variationnelle

## CuPy Optimizations (from CLAUDE.md)

- ❌ `M.tolil()` → Build COO directly
- ❌ `which='SM'` → Use `which='SA'` (Smallest Algebraic)
- ✅ `cp.get_default_memory_pool().free_all_blocks()` for cleanup

---

**Runtime**: ~20-40 min on A100

In [ ]:
# §1 Imports & GPU Setup
import numpy as np
import json
import time
import gc
import os
from datetime import datetime

os.makedirs('outputs', exist_ok=True)

# CuPy (GPU arrays)
try:
    import cupy as cp
    from cupyx.scipy.sparse import csr_matrix as cp_csr
    from cupyx.scipy.sparse.linalg import eigsh as cp_eigsh
    GPU = True
    props = cp.cuda.runtime.getDeviceProperties(0)
    GPU_NAME = props['name'].decode()
    GPU_MEM = props['totalGlobalMem'] / 1e9
    print(f"✓ GPU: {GPU_NAME} ({GPU_MEM:.1f} GB)")
except ImportError:
    GPU = False
    print("⚠ CuPy not available")

# PyTorch (PINN)
import torch
import torch.nn as nn
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.set_default_dtype(torch.float64)
print(f"✓ PyTorch: {DEVICE}")

In [ ]:
# §2 Memory Management

def clear_gpu():
    """Clear all GPU memory."""
    gc.collect()
    if GPU:
        cp.get_default_memory_pool().free_all_blocks()
        cp.get_default_pinned_memory_pool().free_all_blocks()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()

def mem_status():
    """Print GPU memory usage."""
    if torch.cuda.is_available():
        used = torch.cuda.memory_allocated() / 1e9
        total = torch.cuda.get_device_properties(0).total_memory / 1e9
        print(f"GPU: {used:.2f}/{total:.1f} GB ({100*used/total:.0f}%)")

clear_gpu()
mem_status()

In [ ]:
# §3 K₇ Constants with Pell Structure

class K7:
    """K₇ manifold constants derived from Pell equation."""
    
    # Dimensions
    DIM = 7          # dim(K₇)
    DIM_G2 = 14      # dim(G₂) = 2 × dim(K₇)
    
    # Betti numbers (NOT standard TCS!)
    B2 = 21
    B3 = 77
    H_STAR = B2 + B3 + 1  # = 99
    
    # Pell equation: H*² - D × dim(G₂)² = 1
    D = DIM**2 + 1  # = 50
    PELL_CHECK = H_STAR**2 - D * DIM_G2**2  # Should be 1
    
    # Continued fraction: √50 = [7; 14, 14, ...]
    # First Pell+1 convergent: 99/14 = H*/dim(G₂)
    
    # Metric determinant (topological)
    DET_G = 65 / 32  # = 2.03125
    
    # Spectral targets
    LAMBDA1_14 = DIM_G2 / H_STAR      # = 14/99 ≈ 0.1414
    LAMBDA1_13 = (DIM_G2 - 1) / H_STAR  # = 13/99 ≈ 0.1313
    
    # Pell-optimal TCS ratio
    # From ε² = 99 + 14√50, we get ratio = H*/√(D×dim(G₂)²) = 99/√9800
    PELL_RATIO = H_STAR / np.sqrt(D * DIM_G2**2)  # ≈ 1.0001
    
    # Alternative: from √50 ≈ 7.071
    SQRT_D = np.sqrt(D)
    
    # Fano plane (octonion multiplication)
    FANO = [(1,2,4), (2,3,5), (3,4,6), (4,5,7), (5,6,1), (6,7,2), (7,1,3)]

# Verify Pell
print("═" * 50)
print("K₇ PELL STRUCTURE")
print("═" * 50)
print(f"dim(K₇) = {K7.DIM}")
print(f"dim(G₂) = {K7.DIM_G2} = 2 × {K7.DIM}")
print(f"D = dim(K₇)² + 1 = {K7.D}")
print(f"")
print(f"H* = b₂ + b₃ + 1 = {K7.B2} + {K7.B3} + 1 = {K7.H_STAR}")
print(f"")
print(f"Pell check: {K7.H_STAR}² - {K7.D}×{K7.DIM_G2}² = {K7.PELL_CHECK}")
assert K7.PELL_CHECK == 1, "Pell equation failed!"
print(f"           = 9801 - 9800 = 1 ✓")
print(f"")
print(f"√{K7.D} = {K7.SQRT_D:.6f} = [7; 14̄]")
print(f"")
print(f"Targets:")
print(f"  λ₁ = 14/99 = {K7.LAMBDA1_14:.8f} → λ₁×H* = 14")
print(f"  λ₁ = 13/99 = {K7.LAMBDA1_13:.8f} → λ₁×H* = 13")
print(f"")
print(f"det(g) = {K7.DET_G} = 65/32")
print("═" * 50)

In [ ]:
# §4 G₂ Structure Constants

def build_epsilon():
    """G₂ structure constants from Fano plane."""
    eps = torch.zeros(7, 7, 7, dtype=torch.float64, device=DEVICE)
    for (i, j, k) in K7.FANO:
        i, j, k = i-1, j-1, k-1  # 0-indexed
        eps[i,j,k] = eps[j,k,i] = eps[k,i,j] = 1.0
        eps[i,k,j] = eps[k,j,i] = eps[j,i,k] = -1.0
    return eps

EPSILON = build_epsilon()
print(f"✓ G₂ structure: {int((EPSILON != 0).sum())} non-zero entries")

In [ ]:
# §5 3-form φ and Metric

# Index mapping for 35 antisymmetric components
IDX_MAP = [(i,j,k) for i in range(7) for j in range(i+1,7) for k in range(j+1,7)]

def phi_to_tensor(phi_flat):
    """Convert (N, 35) flat φ to (N, 7, 7, 7) antisymmetric tensor."""
    N = phi_flat.shape[0]
    phi = torch.zeros(N, 7, 7, 7, device=phi_flat.device, dtype=phi_flat.dtype)
    for idx, (i,j,k) in enumerate(IDX_MAP):
        v = phi_flat[:, idx]
        phi[:,i,j,k] = phi[:,j,k,i] = phi[:,k,i,j] = v
        phi[:,i,k,j] = phi[:,k,j,i] = phi[:,j,i,k] = -v
    return phi

def phi_to_metric(phi, target_det=K7.DET_G):
    """Compute metric g_ij = (1/6) φ_ikl φ_jkl, rescaled to target det."""
    # B_ij = φ_ikl φ_jkl
    B = torch.einsum('nikl,njkl->nij', phi, phi)
    # g = B/6, then rescale
    g = B / 6.0
    det_g = torch.linalg.det(g)
    # Rescale to target determinant
    scale = (target_det / det_g.abs().clamp(min=1e-10)) ** (1/7)
    g = g * scale.unsqueeze(-1).unsqueeze(-1)
    return g

print("✓ φ → metric conversion ready")

In [ ]:
# §6 PINN Model

class G2PINN(nn.Module):
    """Physics-Informed NN learning G₂ 3-form."""
    
    def __init__(self, hidden=[512, 512, 512, 256]):
        super().__init__()
        layers = []
        dims = [7] + hidden + [35]
        for i in range(len(dims)-1):
            layers.append(nn.Linear(dims[i], dims[i+1]))
            if i < len(dims)-2:
                layers.append(nn.SiLU())
        self.net = nn.Sequential(*layers)
        self._init_weights()
    
    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight, gain=0.5)
                nn.init.zeros_(m.bias)
    
    def forward(self, x):
        return self.net(x)
    
    def get_metric(self, x):
        phi = phi_to_tensor(self(x))
        return phi_to_metric(phi)

model = G2PINN().to(DEVICE)
n_params = sum(p.numel() for p in model.parameters())
print(f"✓ PINN: {n_params:,} parameters")

In [ ]:
# §7 Quaternionic Sampling (S³ × S³)

def sample_S3(n, device=DEVICE):
    """Uniform sampling on S³ (unit quaternions)."""
    x = torch.randn(n, 4, device=device, dtype=torch.float64)
    return x / x.norm(dim=-1, keepdim=True)

def sample_K7(n, device=DEVICE):
    """
    Sample from K₇ ≈ S¹ × S³ × S³.
    
    Returns:
        coords: (n, 7) coordinates
        q1, q2: (n, 4) quaternions for S³ factors
    """
    # S¹ factor
    theta = 2 * np.pi * torch.rand(n, 1, device=device, dtype=torch.float64)
    
    # Two S³ factors
    q1 = sample_S3(n, device)
    q2 = sample_S3(n, device)
    
    # Stereographic projection S³ → ℝ³
    # This gives 1 + 3 + 3 = 7 coordinates
    q1_3d = q1[:, 1:4] / (1 + q1[:, 0:1].abs() + 1e-8)
    q2_3d = q2[:, 1:4] / (1 + q2[:, 0:1].abs() + 1e-8)
    
    coords = torch.cat([theta, q1_3d, q2_3d], dim=-1)
    return coords, q1, q2

# Test
_c, _q1, _q2 = sample_K7(100)
print(f"✓ Sampling: coords {_c.shape}, q1 {_q1.shape}, q2 {_q2.shape}")

In [ ]:
# §8 Physics Loss

def physics_loss(model, coords, λ_det=10.0, λ_norm=0.1):
    """Combined physics loss for G₂ structure."""
    N = coords.shape[0]
    phi_flat = model(coords)
    phi = phi_to_tensor(phi_flat)
    
    # L1: Torsion (deviation from standard G₂)
    phi_std = EPSILON.unsqueeze(0).expand(N, -1, -1, -1)
    L_torsion = ((phi - phi_std) ** 2).mean()
    
    # L2: Determinant constraint
    g = phi_to_metric(phi)
    det_g = torch.linalg.det(g)
    L_det = ((det_g - K7.DET_G) ** 2).mean()
    
    # L3: Normalization φ_ikl φ_jkl = 6 δ_ij
    B = torch.einsum('nikl,njkl->nij', phi, phi)
    I6 = 6.0 * torch.eye(7, device=coords.device, dtype=torch.float64)
    L_norm = ((B - I6) ** 2).mean()
    
    loss = L_torsion + λ_det * L_det + λ_norm * L_norm
    
    return loss, {
        'total': float(loss),
        'torsion': float(L_torsion),
        'det': float(L_det),
        'det_mean': float(det_g.mean())
    }

print("✓ Physics loss defined")

In [ ]:
# §9 PINN Training

def train_pinn(model, epochs=5000, batch=8192, lr=1e-3):
    """Train PINN with cosine annealing."""
    opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-5)
    sched = torch.optim.lr_scheduler.CosineAnnealingLR(opt, epochs, eta_min=1e-6)
    
    history = {'loss': [], 'det': []}
    best_loss = float('inf')
    
    t0 = time.time()
    for ep in range(epochs):
        coords, _, _ = sample_K7(batch)
        
        opt.zero_grad()
        loss, info = physics_loss(model, coords)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        opt.step()
        sched.step()
        
        history['loss'].append(info['total'])
        history['det'].append(info['det_mean'])
        
        if info['total'] < best_loss:
            best_loss = info['total']
            torch.save(model.state_dict(), 'outputs/k7_pinn_best.pt')
        
        if ep % 500 == 0 or ep == epochs - 1:
            print(f"Ep {ep:5d} | Loss {info['total']:.2e} | "
                  f"det(g) {info['det_mean']:.6f} | {time.time()-t0:.1f}s")
        
        if ep % 1000 == 0:
            clear_gpu()
    
    print(f"\n✓ Training done, best loss: {best_loss:.2e}")
    return history

print("═" * 50)
print("PHASE 1: PINN TRAINING")
print("═" * 50)

history = train_pinn(model, epochs=5000)

# Load best
model.load_state_dict(torch.load('outputs/k7_pinn_best.pt'))
model.eval()
clear_gpu()

In [ ]:
# §10 High-Resolution Point Cloud

N = 100000  # High resolution

print(f"Sampling N = {N:,} points...")

with torch.no_grad():
    coords_all, q1_all, q2_all = sample_K7(N)
    
    # Get metrics in batches
    g_list = []
    batch = 10000
    for i in range(0, N, batch):
        g_batch = model.get_metric(coords_all[i:i+batch])
        g_list.append(g_batch.cpu())
        if i % 20000 == 0:
            clear_gpu()
    g_all = torch.cat(g_list, dim=0)

# To numpy
coords = coords_all.cpu().numpy()
g = g_all.numpy()
q1 = q1_all.cpu().numpy()
q2 = q2_all.cpu().numpy()

# Verify det(g)
det_g = np.linalg.det(g)
print(f"\n✓ N = {N:,} points")
print(f"  det(g) = {det_g.mean():.8f} ± {det_g.std():.2e}")
print(f"  target = {K7.DET_G:.8f}")
print(f"  error  = {abs(det_g.mean() - K7.DET_G):.2e}")

clear_gpu()
mem_status()

In [ ]:
# §11 Geodesic Distance on S³

def geodesic_S3(q1, q2):
    """Geodesic distance between quaternions on S³."""
    dot = np.abs(np.sum(q1 * q2, axis=-1))
    dot = np.clip(dot, 0, 1)
    return np.arccos(dot)

print("✓ Geodesic distance ready")

In [ ]:
# §12 Build Laplacian with CuPy (COO format)

def build_laplacian_cupy(q1, q2, g, k=50):
    """
    Build Laplace-Beltrami matrix using CuPy.
    
    Key: Build COO directly (no tolil()!)
    """
    N = len(q1)
    print(f"Building Laplacian (N={N:,}, k={k})...")
    
    # Volume weights: √det(g)
    det_g = np.linalg.det(g)
    sqrt_det = np.sqrt(np.abs(det_g))
    
    # COO format lists
    rows, cols, data = [], [], []
    
    t0 = time.time()
    batch_size = 2000
    
    for i_start in range(0, N, batch_size):
        i_end = min(i_start + batch_size, N)
        B = i_end - i_start
        
        # Geodesic distances on S³ × S³
        d1 = geodesic_S3(q1[i_start:i_end, None, :], q1[None, :, :])  # (B, N)
        d2 = geodesic_S3(q2[i_start:i_end, None, :], q2[None, :, :])  # (B, N)
        
        # Combined distance with Pell-optimal weighting
        # Use H* and dim(G₂) from Pell structure
        r1 = K7.H_STAR / K7.SQRT_D  # ≈ 14.0
        r2 = K7.DIM_G2              # = 14
        dist_sq = (r1 * d1)**2 + (r2 * d2)**2
        
        for local_i in range(B):
            global_i = i_start + local_i
            dists = dist_sq[local_i].copy()
            dists[global_i] = np.inf  # Exclude self
            
            # k nearest neighbors
            nn_idx = np.argpartition(dists, k)[:k]
            nn_dists = np.sqrt(dists[nn_idx])
            
            # Heat kernel with volume weighting
            sigma = np.median(nn_dists) + 1e-8
            weights = np.exp(-nn_dists**2 / (2 * sigma**2))
            weights *= sqrt_det[global_i] * sqrt_det[nn_idx]
            weights /= (nn_dists + 1e-8)
            weights /= (weights.sum() + 1e-10)
            
            # Add to COO
            for j, w in zip(nn_idx, weights):
                rows.append(global_i)
                cols.append(int(j))
                data.append(-float(w))
            
            # Diagonal
            rows.append(global_i)
            cols.append(global_i)
            data.append(float(weights.sum()))
        
        if (i_start // batch_size) % 10 == 0:
            pct = 100 * i_end / N
            eta = (time.time() - t0) / (i_end / N) * (1 - i_end / N)
            print(f"  {pct:.0f}% | ETA {eta:.0f}s")
    
    # Build CuPy sparse matrix from COO
    rows = cp.array(rows, dtype=cp.int32)
    cols = cp.array(cols, dtype=cp.int32)
    data = cp.array(data, dtype=cp.float64)
    
    L = cp_csr((data, (rows, cols)), shape=(N, N))
    
    # Symmetrize
    L = (L + L.T) / 2
    
    print(f"\n✓ Laplacian built in {time.time()-t0:.1f}s")
    print(f"  Shape: {L.shape}, nnz: {L.nnz:,}")
    
    return L

print("✓ CuPy Laplacian builder ready")

In [ ]:
# §13 Build Laplacian

print("═" * 50)
print("PHASE 2: LAPLACIAN CONSTRUCTION")
print("═" * 50)

clear_gpu()

L = build_laplacian_cupy(q1, q2, g, k=50)

clear_gpu()
mem_status()

In [ ]:
# §14 Eigenvalue Computation (CuPy)

print("═" * 50)
print("PHASE 3: SPECTRAL COMPUTATION")
print("═" * 50)

n_eigs = 30
print(f"Computing {n_eigs} smallest eigenvalues...")

t0 = time.time()

# Use 'SA' (Smallest Algebraic), NOT 'SM'!
eigenvalues, eigenvectors = cp_eigsh(L, k=n_eigs, which='SA')

# Sort and convert to numpy
eigenvalues = cp.sort(eigenvalues).get()

print(f"\n✓ Computed in {time.time()-t0:.1f}s")

print("\nSpectrum:")
print("─" * 40)
for i, ev in enumerate(eigenvalues[:15]):
    ev_Hstar = ev * K7.H_STAR
    print(f"  λ_{i:2d} = {ev:12.8f}  |  λ×H* = {ev_Hstar:8.4f}")

In [ ]:
# §15 Spectral Gap Analysis

print("═" * 50)
print("SPECTRAL GAP ANALYSIS")
print("═" * 50)

# Find spectral gap (first positive eigenvalue)
lambda_0 = eigenvalues[0]
lambda_1 = eigenvalues[1]

# If λ₀ ≈ 0, use λ₁
if abs(lambda_0) < 0.01:
    gap = lambda_1
    print(f"λ₀ = {lambda_0:.8f} ≈ 0 (constant mode) ✓")
else:
    # Use first positive
    pos = eigenvalues[eigenvalues > 0.001]
    gap = pos[0] if len(pos) > 0 else lambda_1
    print(f"λ₀ = {lambda_0:.8f} (using first positive)")

gap_Hstar = gap * K7.H_STAR

print(f"\nSpectral gap λ₁ = {gap:.8f}")
print(f"")
print(f"λ₁ × H* = {gap_Hstar:.4f}")
print(f"")
print(f"Comparison:")
print(f"  Target 14: deviation = {abs(gap_Hstar - 14) / 14 * 100:.2f}%")
print(f"  Target 13: deviation = {abs(gap_Hstar - 13) / 13 * 100:.2f}%")
print(f"")

# Determine which is closer
dev_14 = abs(gap_Hstar - 14)
dev_13 = abs(gap_Hstar - 13)

if dev_14 < dev_13:
    result = 14
    print(f"→ Closer to λ₁ × H* = 14 (GIFT prediction)")
else:
    result = 13
    print(f"→ Closer to λ₁ × H* = 13 (dim(G₂) - 1)")

print(f"")
if min(dev_14, dev_13) < 1:
    print(f"✓ VALIDATION PASSED (< 8% error)")
elif min(dev_14, dev_13) < 2:
    print(f"◐ PARTIAL (< 15% error)")
else:
    print(f"✗ Needs refinement")

In [ ]:
# §16 Pell Structure Verification

print("═" * 50)
print("PELL STRUCTURE CHECK")
print("═" * 50)

# If λ₁ × H* ≈ 14, verify Pell relations
print(f"\nAlgebraic verification:")
print(f"  H*² = {K7.H_STAR**2}")
print(f"  50 × 14² = {50 * 14**2}")
print(f"  Difference = {K7.H_STAR**2 - 50 * 14**2} (should be 1)")
print(f"")
print(f"Continued fraction [0; 7, 14]:")
cf_val = 1 / (7 + 1/14)
print(f"  1/(7 + 1/14) = {cf_val:.10f}")
print(f"  14/99       = {14/99:.10f}")
print(f"  Match: {abs(cf_val - 14/99) < 1e-10}")
print(f"")
print(f"Numerical result:")
print(f"  λ₁ × H* = {gap_Hstar:.6f}")
print(f"  Nearest integer: {round(gap_Hstar)}")

In [ ]:
# §17 Export Results

results = {
    'timestamp': datetime.now().isoformat(),
    'gpu': GPU_NAME if GPU else 'CPU',
    'n_points': int(N),
    'k_neighbors': 50,
    
    'pell': {
        'H_star': int(K7.H_STAR),
        'dim_G2': int(K7.DIM_G2),
        'D': int(K7.D),
        'check': int(K7.PELL_CHECK),
    },
    
    'metric': {
        'det_mean': float(det_g.mean()),
        'det_std': float(det_g.std()),
        'det_target': float(K7.DET_G),
    },
    
    'spectral': {
        'eigenvalues': [float(e) for e in eigenvalues.tolist()],
        'lambda_1': float(gap),
        'lambda1_times_Hstar': float(gap_Hstar),
        'closest_target': int(result),
        'deviation_14_pct': float(dev_14 / 14 * 100),
        'deviation_13_pct': float(dev_13 / 13 * 100),
    },
    
    'conclusion': {
        'lambda1_times_Hstar': float(gap_Hstar),
        'closest_integer': int(round(gap_Hstar)),
        'supports_14': bool(dev_14 < dev_13),
    }
}

with open('outputs/k7_pinn_cupy_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("✓ Results saved: outputs/k7_pinn_cupy_results.json")

In [ ]:
# §18 Final Summary

print("")
print("╔" + "═" * 58 + "╗")
print("║" + " K₇ EXPLICIT CONSTRUCTION — FINAL RESULTS ".center(58) + "║")
print("╠" + "═" * 58 + "╣")
print("║" + f"  Pell equation: 99² - 50×14² = {K7.PELL_CHECK}".ljust(58) + "║")
print("║" + f"  √50 = [7; 14̄] = dim(K₇); dim(G₂)...".ljust(58) + "║")
print("╠" + "═" * 58 + "╣")
print("║" + f"  det(g) = {det_g.mean():.8f} (target: {K7.DET_G})".ljust(58) + "║")
print("║" + f"  error  = {abs(det_g.mean() - K7.DET_G):.2e}".ljust(58) + "║")
print("╠" + "═" * 58 + "╣")
print("║" + f"  λ₁ × H* = {gap_Hstar:.4f}".ljust(58) + "║")
print("║" + f"  Closest to: {result} (deviation {min(dev_14, dev_13)/result*100:.1f}%)".ljust(58) + "║")
print("╠" + "═" * 58 + "╣")

if min(dev_14, dev_13) < 1:
    status = "✓ CONSTRUCTION VALIDATED"
elif min(dev_14, dev_13) < 2:
    status = "◐ PARTIAL VALIDATION"
else:
    status = "✗ NEEDS REFINEMENT"

print("║" + f"  {status}".ljust(58) + "║")
print("╚" + "═" * 58 + "╝")